In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sidetable

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer


Trabajamos para una inmobiliaria, nuestros jefes han obtenido una serie de datos sobre los inmuebles que están en alquiler en toda España. Ellos están iniciando su negocio y quieren focalizarse en aquellas comunidades autónomas donde más movimiento de compra-venta de inmuebles. 

Ojo!! Solo les interesa los inmuebles para vivir, piso, chalet, etc. 

En concreto, quieren identificar las 5 CCAA que tienen más inmuebles en alquiler y contestar a una serie de preguntas:

- ¿Cuál es el precio medio por tipo de inmueble en función de la provincia?
- ¿Qué tipo de inmuebles son los que más se ofertan?

In [3]:
df = pd.read_csv("casas2.csv", index_col = 0)
df.head(3)

,HY_id,HY_cod_postal,HY_provincia,HY_descripcion,HY_distribucion,HY_antiguedad,HY_metros_utiles,HY_metros_totales,HY_num_banos,HY_cert_energ,HY_num_terrazas,HY_ascensor,HY_trastero,HY_num_garajes,HY_precio,HY_precio_anterior,IDEA_area,IDEA_poblacion,IDEA_densidad
0,6028499,18151,Granada,GARAJE-Plaza de Garaje ubicada en la planta só...,NaN,NaN,NaN,"35,0",0,NaN,0,0,0,0,12300.0,NaN,670.87,"13372,0",19.93
1,6028500,29915,Málaga,CASA DE PUEBLO,NaN,NaN,NaN,"151,98",2,NaN,0,0,0,0,115000.0,125000.0,2216.24,"1507,0",0.68
2,6028502,3740,Alicante,PISO-El inmueble(piso) se encuentra situado en...,VIVIENDA-PISO,2008.0,92.58,"102,78",1,NaN,0,1,0,1,88000.0,93500.0,2040.53,"5810,0",2.85


In [6]:
nuevas_columnas ={col: "_".join(col.split('_')[1:]) for col in df.columns}
df.rename(columns=nuevas_columnas, inplace = True)

In [7]:
df.head(2)

,id,cod_postal,provincia,descripcion,distribucion,antiguedad,metros_utiles,metros_totales,num_banos,cert_energ,num_terrazas,ascensor,trastero,num_garajes,precio,precio_anterior,area,poblacion,densidad
0,6028499,18151,Granada,GARAJE-Plaza de Garaje ubicada en la planta só...,NaN,NaN,NaN,"35,0",0,NaN,0,0,0,0,12300.0,NaN,670.87,"13372,0",19.93
1,6028500,29915,Málaga,CASA DE PUEBLO,NaN,NaN,NaN,"151,98",2,NaN,0,0,0,0,115000.0,125000.0,2216.24,"1507,0",0.68


In [9]:
df['metros_totales'] = df['metros_totales'].str.replace(',','.')#no puede hacerse para todas las columnas a la vez

In [10]:
df['poblacion'] = df['poblacion'].str.replace(',','.')

In [ ]:
columnas_cambiar =['metros_totales', 'poblacion'] #Para cambiarlo de todas las columnas que tenemos que cambiar la , por el punto 
for i in columnas_cambiar:
    df[i] = df[i].str.replace(',','.')

In [11]:
df.dtypes #Siguen siendo tipo object

id                   int64
cod_postal           int64
provincia           object
descripcion         object
distribucion        object
antiguedad         float64
metros_utiles      float64
metros_totales      object
num_banos            int64
cert_energ          object
num_terrazas         int64
ascensor             int64
trastero             int64
num_garajes          int64
precio             float64
precio_anterior    float64
area               float64
poblacion           object
densidad           float64
dtype: object

In [12]:
df['metros_totales'] = pd.to_numeric(df['metros_totales']) #no puede hacerse para todas las columnas a la vez
df['poblacion'] = pd.to_numeric(df['poblacion'])

In [13]:
df.head()

,id,cod_postal,provincia,descripcion,distribucion,antiguedad,metros_utiles,metros_totales,num_banos,cert_energ,num_terrazas,ascensor,trastero,num_garajes,precio,precio_anterior,area,poblacion,densidad
0,6028499,18151,Granada,GARAJE-Plaza de Garaje ubicada en la planta só...,NaN,NaN,NaN,35.00,0,NaN,0,0,0,0,12300.0,NaN,670.87,13372.0,19.93
1,6028500,29915,Málaga,CASA DE PUEBLO,NaN,NaN,NaN,151.98,2,NaN,0,0,0,0,115000.0,125000.0,2216.24,1507.0,0.68
2,6028502,3740,Alicante,PISO-El inmueble(piso) se encuentra situado en...,VIVIENDA-PISO,2008.0,92.58,102.78,1,NaN,0,1,0,1,88000.0,93500.0,2040.53,5810.0,2.85
3,6028512,30592,Murcia,GARAJE-Plaza de garaje situada en la planta se...,NaN,NaN,11.81,12.46,0,NaN,0,1,0,0,5800.0,NaN,905.44,7004.0,7.74
4,6028513,30592,Murcia,GARAJE-Plaza de garaje situada en la planta se...,NaN,NaN,14.29,15.43,0,NaN,0,1,0,0,7400.0,NaN,905.44,7004.0,7.74


In [30]:
dicc_comunidades = {"Andalucía" : ['Almería', 'Cádiz', 'Córdoba', 'Granada', 'Huelva', 'Jaén', 'Málaga', 'Sevilla'], 
                    'Aragón' : ['Huesca', 'Teruel', 'Zaragoza'], 
                    'Asturias' : ["Asturias"], 
                    "Baleares" : ["Baleares"], 
                    'Canarias' : ['Tenerife', 'Las Palmas'], 
                    'Cantabria' : ["Cantabria"], 
                   'Castilla-La Mancha' : ['Albacete', 'Ciudad Real', 'Cuenca', 'Guadalajara', 'Toledo'], 
                   'Castilla y León' : ['Avila', 'Burgos', 'León', 'Salamanca', 'Segovia', 'Soria', 'Valladolid', 'Zamora', 'Palencia'], 
                   'Cataluña' : ['Barcelona', 'Gerona', 'Lérida', 'Tarragona'], 
                   'Comunidad Valenciana' : ['Alicante', 'Castellón', 'Valencia'], 
                   'Extremadura' : ['Badajoz', 'Cáceres'], 
                   'Galicia' :[ 'A Coruña', 'Lugo', 'Ourense', 'Pontevedra'], 
                   "Madrid": ["Madrid"], 
                   "Murcia": ["Murcia"], 
                   "Navarra": ["Navarra"], 
                   'País Vasco' :[ 'Vizkaia', 'Guipuzkua', 'Alava'], 
                   "La Rioja": ["La Rioja"]
}

In [24]:
def sacar_comunidades(col, diccionario):
    for k,v in diccionario.items():
        if col in v:
            return k
    return np.nan #Para evitar que nos salgan None al encontrar nan

In [25]:
df['comunidad'] =df.apply(lambda data: sacar_comunidades(data['provincia'], dicc_comunidades), axis = 1) #Para aplicar una funcion a un df con varios parametros

In [26]:
df['comunidad'].unique()

array(['Andalucía', 'Comunidad Valenciana', 'Murcia', 'Cataluña',
       'Castilla y León', 'Baleares', 'Canarias', 'Extremadura', 'Madrid',
       'Castilla-La Mancha', 'Aragón', 'La Rioja', 'Navarra', 'Asturias',
       'Galicia', nan], dtype=object)

In [29]:
df.isnull().sum()

id                    0
cod_postal            0
provincia            42
descripcion           0
distribucion       6745
antiguedad         5484
metros_utiles      2886
metros_totales       34
num_banos             0
cert_energ         8300
num_terrazas          0
ascensor              0
trastero              0
num_garajes           0
precio                0
precio_anterior    3158
area               2631
poblacion          2635
densidad           2635
comunidad            42
dtype: int64

In [31]:
!pip install geopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.9 MB/s eta 0:00:00


In [ ]:
#Usando una libreria de provincias
from geopy.geocoders import Nominatim



In [34]:
#Sacar las 5 comunidades con mas inmuebles
lista_comunidades = df['comunidad'].value_counts()[:5].index.tolist()

In [35]:
df2 = df[df['comunidad'].isin(lista_comunidades)]
df2['comunidad'].unique()

array(['Andalucía', 'Comunidad Valenciana', 'Murcia', 'Cataluña',
       'Castilla y León'], dtype=object)

In [ ]:
plt.tight_layout() #Para autoajustar las grafias al tamaño de figura
axes = axes.flat 
fig.delaxes(axes[-1]) # Para eliminar el último grafico si esta vacio